In [3]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

In [5]:
from match_elements.GUI_pair import GUIPair
from GUI import GUI

In [3]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [2]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

[2022/04/27 17:47:41] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/27 17:47:41] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/27 17:47:41] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [7]:
from ppadb.client import Client as AdbClient
client = AdbClient(host="127.0.0.1", port=5037)

In [9]:
def cap_device_current_gui(device, paddle_ocr, screen_path="data/output/screen.jpg", is_detect=True):
    screen = device.screencap()
    with open(screen_path, "wb") as fp:
        fp.write(screen)
        
    gui = GUI(screen_path)
    if is_detect:
        gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
    else:
        gui.load_detection_result()
    return gui

In [7]:
def match_element_in_gui(target_element, gui, device, resnet_model, scroll_down=True):
    matched_ele = gui.match_elements(target_element.clip, resnet_model, target_element.text_content, min_similarity_img=0.55, show=True)
    # scroll down if current gui is not matched
    if matched_ele is None and scroll_down:
        print('Scroll down and try to match again')
        # device.shell('input roll' + str(device.wm_size().height - 50) + ' 0')
        device.input_swipe(50, (device.wm_size().height * 0.9), 50, 20, 500)
        next_gui = cap_device_current_gui(device, paddle_ocr)
        # cv2.imshow('img', next_gui.img)
        # next_gui.show_detection_result()
        match_element_in_gui(target_element, next_gui, device, resnet_model, scroll_down=False)
    return matched_ele

In [10]:
dev1 = client.device('emulator-5554')
gui1 = cap_device_current_gui(dev1, paddle_ocr, screen_path='data/screen/80.png', is_detect=True)
gui1.show_detection_result()
# gui1 = GUI('data/screen/80.png')
# gui1.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui1.load_detection_result()

[Text Detection Completed in 0.577 s] Input: data/screen/80.png Output: data/output/ocr/80.json
[Compo Detection Completed in 0.387 s] Input: data/screen/80.png Output: data/output/ip/80.json
[Merge Completed] Input: data/screen/80.png Output: data/output/merge/80.jpg


In [9]:
dev2 = client.device('emulator-5558')
gui2 = cap_device_current_gui(dev2, paddle_ocr, screen_path='data/screen/60.png', is_detect=True)
gui2.show_detection_result()

[2022/04/25 15:03:01] root DEBUG: dt_boxes num : 35, elapse : 0.1888594627380371
[2022/04/25 15:03:01] root DEBUG: cls num  : 35, elapse : 0.12176203727722168
[2022/04/25 15:03:01] root DEBUG: rec_res num  : 35, elapse : 0.1688835620880127
[Text Detection Completed in 0.557 s] Input: data/screen/60.png Output: data/output/ocr/60.json
[Compo Detection Completed in 0.454 s] Input: data/screen/60.png Output: data/output/ip/60.json
[Merge Completed] Input: data/screen/60.png Output: data/output/merge/60.jpg


In [10]:
dev3 = client.device('emulator-5560')
gui3 = cap_device_current_gui(dev3, paddle_ocr, screen_path='data/screen/50.png', is_detect=True)
gui3.show_detection_result()

[2022/04/25 15:03:06] root DEBUG: dt_boxes num : 28, elapse : 0.16089081764221191
[2022/04/25 15:03:06] root DEBUG: cls num  : 28, elapse : 0.09769868850708008
[2022/04/25 15:03:06] root DEBUG: rec_res num  : 28, elapse : 0.15407514572143555
[Text Detection Completed in 0.482 s] Input: data/screen/50.png Output: data/output/ocr/50.json
[Compo Detection Completed in 0.540 s] Input: data/screen/50.png Output: data/output/ip/50.json
[Merge Completed] Input: data/screen/50.png Output: data/output/merge/50.jpg


In [11]:
dev4 = client.device('emulator-5556')
gui4 = cap_device_current_gui(dev4, paddle_ocr, screen_path='data/screen/37.png', is_detect=True)
gui4.show_detection_result()

[2022/04/25 15:03:11] root DEBUG: dt_boxes num : 20, elapse : 0.11971807479858398
[2022/04/25 15:03:11] root DEBUG: cls num  : 20, elapse : 0.07053303718566895
[2022/04/25 15:03:12] root DEBUG: rec_res num  : 20, elapse : 0.1258535385131836
[Text Detection Completed in 0.331 s] Input: data/screen/37.png Output: data/output/ocr/37.json
[Compo Detection Completed in 0.382 s] Input: data/screen/37.png Output: data/output/ip/37.json
[Merge Completed] Input: data/screen/37.png Output: data/output/merge/37.jpg


In [12]:
e = gui2.get_element_by_clicking()

Click (107, 739)


In [13]:
matched_e1 = match_element_in_gui(e, gui1, dev1, resnet_model)
# matched_e2 = match_element_in_gui(e, gui2, dev2, resnet_model)
matched_e3 = match_element_in_gui(e, gui3, dev3, resnet_model)
matched_e4 = match_element_in_gui(e, gui4, dev4, resnet_model)

Match by text
No matched element found
Scroll down and try to match again
[2022/04/25 15:03:59] root DEBUG: dt_boxes num : 27, elapse : 0.16750288009643555
[2022/04/25 15:03:59] root DEBUG: cls num  : 27, elapse : 0.09422802925109863
[2022/04/25 15:03:59] root DEBUG: rec_res num  : 27, elapse : 0.13907909393310547
[Text Detection Completed in 0.468 s] Input: data/output/screen.jpg Output: data/output/ocr/screen.json
[Compo Detection Completed in 0.557 s] Input: data/output/screen.jpg Output: data/output/ip/screen.json
[Merge Completed] Input: data/output/screen.jpg Output: data/output/merge/screen.jpg
Match by text
No matched element found
Scroll down and try to match again
[2022/04/25 15:04:05] root DEBUG: dt_boxes num : 19, elapse : 0.12806248664855957
[2022/04/25 15:04:05] root DEBUG: cls num  : 19, elapse : 0.06684446334838867
[2022/04/25 15:04:05] root DEBUG: rec_res num  : 19, elapse : 0.113922119140625
[Text Detection Completed in 0.327 s] Input: data/output/screen.jpg Output: d

In [21]:
dev1.wm_size()

Size(width=2200, height=2480)

In [22]:
dev2.wm_size()

Size(width=1080, height=2280)

In [23]:
dev3.wm_size()

Size(width=1080, height=1920)

In [24]:
dev4.wm_size()

Size(width=480, height=800)